In [21]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
import tracker
latest_date = tracker.processing_dates[-1]
os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [22]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/fastq/{}.fastq.samplesheet.with_header.tsv'.format(latest_date)
    output_prefix = 'results/samplesheets/fastq/{}.fastq.post-check.download_srr_fastqs'.format(latest_date)
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

In [23]:
latest_date

'2022.04.09.16.57'

## Helper Functions

In [24]:
def list_to_str(l, sep=' '):
    l = [str(x) for x in l]
    return(sep.join(l))

def read_log(log):
    
    """
    Read the log and extract status information including
    the sample_name, srr_id, status (as indicated by the end message),
    and the download status (as indicate by the "reads written" message).
    """
    
    status = 0
    download = 1
    sample_name = 'check'
    srr_id = 'check'
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            elif 'srr_id' in info:
                srr_id = info.split()[1]
            elif 'Ended: download_srr_fastqs' in info:
                status = 1
            elif 'reads written' in info:
                download = 1
            
    return([index, sample_name, srr_id, job_id, status, download])

def long_to_short_index_list(serial_list):
    
    small_num = serial_list[0]
    serial_list_short = []
    for i in range(1, len(serial_list)):

        prev_num = serial_list[i -1]
        next_num = serial_list[i]

        if (prev_num + 1) != next_num:

            if small_num == prev_num:
                curr_range = '{}'.format(small_num)
                serial_list_short.append(curr_range)
                small_num = next_num

            else:
                curr_range = '{}-{}'.format(small_num, prev_num)
                serial_list_short.append(curr_range)
                small_num = next_num

    if small_num == next_num:
        curr_range = '{}'.format(small_num,)
        serial_list_short.append(curr_range)
    else:
        curr_range = '{}-{}'.format(small_num, next_num)
        serial_list_short.append(curr_range)
        
    return(serial_list_short)

## Check Logs

In [25]:
# load the data
df = pd.read_table(input_fn)
df['sample_index'] = df.index + 1 # add 1 since we'll be using 1-based indexing with sed

In [26]:
df

,std_sample_name,gse_id,gsm_id,srr_id,organism,bio_rep,tech_rep,antibody_target,restriction_enzyme,sample_name,sample_index
0,mES_25m_cells.GSE101498.Mus_Musculus.H3K27ac.b1,GSE101498,GSM2705031,SRR5831479,Mus_Musculus,1,1,H3K27ac,MboI,mES_25m_cells,1
1,mES_25m_cells.GSE101498.Mus_Musculus.H3K27ac.b1,GSE101498,GSM2705032,SRR5831480,Mus_Musculus,1,2,H3K27ac,MboI,mES_25m_cells,2
2,mES_25m_cells.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,GSM2705033,SRR5831481,Mus_Musculus,2,1,H3K27ac,MboI,mES_25m_cells,3
3,mES_25m_cells.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,GSM2705034,SRR5831482,Mus_Musculus,2,2,H3K27ac,MboI,mES_25m_cells,4
4,mES_500k_cells.GSE101498.Mus_Musculus.H3K27ac.b1,GSE101498,GSM2705035,SRR5831483,Mus_Musculus,1,1,H3K27ac,MboI,mES_500k_cells,5
5,mES_500k_cells.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,GSM2705036,SRR5831484,Mus_Musculus,2,1,H3K27ac,MboI,mES_500k_cells,6
6,mES_100k_cells.GSE101498.Mus_Musculus.H3K27ac.b1,GSE101498,GSM2705037,SRR5831485,Mus_Musculus,1,1,H3K27ac,MboI,mES_100k_cells,7
7,mES_100k_cells.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,GSM2705038,SRR5831486,Mus_Musculus,2,1,H3K27ac,MboI,mES_100k_cells,8
8,mES_50k_cells.GSE101498.Mus_Musculus.H3K27ac.b1,GSE101498,GSM2705039,SRR5831487,Mus_Musculus,1,1,H3K27ac,MboI,mES_50k_cells,9
9,mES_50k_cells.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,GSM2705040,SRR5831488,Mus_Musculus,2,1,H3K27ac,MboI,mES_50k_cells,10


In [27]:
# create a dataframe of log information 
logs = glob.glob('results/fastqs/raw/logs/download_srr_fastqs.o*-*')
log_data = []
for log in logs:
    index, sample_name, srr_id, job_id, status, download = read_log(log)
    log_data.append([index, sample_name, srr_id, job_id, status, download, log])
log_df = pd.DataFrame(log_data)
log_df.columns = ['sample_index', 'std_sample_name', 'srr_id', 'job_id', 'end_status', 'read_written_status', 'log']
log_df.drop('sample_index', inplace=True, axis=1)

In [28]:
# merge the log information
df = df.merge(log_df, how='left', on=['std_sample_name', 'srr_id'], indicator='merge')
df.loc[df.end_status.isna(), 'end_status'] = 0
df.loc[df.read_written_status.isna(), 'read_written_status'] = 0
df.end_status = df.end_status.astype(int)
df.read_written_status = df.read_written_status.astype(int)

In [29]:
# get the latest job id for a give sample + srr id combo
df = df.sort_values('job_id', ascending=False).drop_duplicates(['std_sample_name', 'srr_id'])

# sort back into the sample index values, needed for correctly running qsub
df.sort_values('sample_index', inplace=True)

## Check the presences of SRR FASTQ files

In [30]:
# get the download path
download_tpl = 'results/fastqs/raw/{}/{}_2.fastq.gz'
df.loc[:, 'download_output'] = df.apply(lambda x: download_tpl.format(x[0], x[3]), axis=1)

# find whether the download SRR FASTQ is present
df.loc[:, 'download_present'] = df.loc[:, 'download_output'].apply(os.path.exists).astype(int)

In [31]:
reorder_cols = ['std_sample_name',
                 'gse_id',
                 'gsm_id',
                 'srr_id',
                 'organism',
                 'bio_rep',
                 'tech_rep',
                 'sample_index',
                 'job_id',
                 'end_status',
                 'read_written_status',
                 'download_present',
                 'antibody_target',
                 'restriction_enzyme',
                 'sample_name',
                 'log',
                 'download_output']

In [32]:
df = df[reorder_cols]

In [33]:
save_df = df.rename(columns={'end_status': 'download_end_status', 
                   'read_written_status': 'download_read_written_status',
                   'log': 'download_log', 
                   'job_id': 'download_job_id'})

header_output = '{}.with_header.tsv'.format(output_prefix)
save_df.to_csv(header_output, header=True, index=False, sep='\t')

without_header_output = '{}.without_header.tsv'.format(output_prefix)
save_df.to_csv(without_header_output, header=False, index=False, sep='\t')

## Identifying problem samples

In [34]:
problems = (df.end_status != 1) | (df.read_written_status != 1) | (df.download_present != 1)
problems_df = df.loc[problems,:]
problems_df

,std_sample_name,gse_id,gsm_id,srr_id,organism,bio_rep,tech_rep,sample_index,job_id,end_status,read_written_status,download_present,antibody_target,restriction_enzyme,sample_name,log,download_output
119,HCT116-AuxinNeg.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,GSE179544,GSM5420289,SRR15050696,Homo_Sapiens,1,1,120,4932633,0,1,0,RNA-Pol-II,DpnII,HCT116-AuxinNeg,results/fastqs/raw/logs/download_srr_fastqs.o4932633-120,results/fastqs/raw/HCT116-AuxinNeg.GSE179544.Homo_Sapiens.RNA-Pol-II.b1/SRR15050696_2.fastq.gz
123,HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,GSE179544,GSM5420290,SRR15050697,Homo_Sapiens,1,1,122,4932633,0,1,0,RNA-Pol-II,DpnII,HCT116-AuxinPos,results/fastqs/raw/logs/download_srr_fastqs.o4932633-122,results/fastqs/raw/HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1/SRR15050697_2.fastq.gz
124,HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,GSE179544,GSM5420292,SRR15050699,Homo_Sapiens,1,2,123,4932633,0,1,0,RNA-Pol-II,DpnII,HCT116-AuxinPos,results/fastqs/raw/logs/download_srr_fastqs.o4932633-123,results/fastqs/raw/HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1/SRR15050699_2.fastq.gz


In [35]:
problems_df.loc[:, ['std_sample_name', 'log']]

,std_sample_name,log
119,HCT116-AuxinNeg.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,results/fastqs/raw/logs/download_srr_fastqs.o4932633-120
123,HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,results/fastqs/raw/logs/download_srr_fastqs.o4932633-122
124,HCT116-AuxinPos.GSE179544.Homo_Sapiens.RNA-Pol-II.b1,results/fastqs/raw/logs/download_srr_fastqs.o4932633-123


In [36]:
problems_df.shape

(3, 17)

## Rerun problematic samples

In [37]:
# convert not run serial IDs to ranges/short format
rerun_indexes = sorted(problems_df.sample_index.tolist())
rerun_indexes = long_to_short_index_list(rerun_indexes)

In [38]:
rerun_indexes

['120', '122-123']

In [39]:
# generate a string version of not_run
final_rerun = ','.join(rerun_indexes)
final_rerun = 'qsub -t {} workflow/scripts/fastq/download_srr_fastqs.qarray.qsh'.format(final_rerun)

In [40]:
print(final_rerun)

qsub -t 120,122-123 workflow/scripts/fastq/download_srr_fastqs.qarray.qsh


<p style="color: red"><b>Re-running in progress: 2022.04.02 - 16:30</b></p>